In [1]:
!pip install category_encoders

In [0]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My\ Drive/Datasets/Janata_hack

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Datasets/Janata_hack


In [0]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
train_df.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [6]:
train_df.describe()

,session_id,startTime,endTime,ProductList,gender
count,10500,10500,10500,10500,10500
unique,10500,8815,8803,9402,2
top,u17210,17/12/14 12:29,14/11/14 16:15,A00002/B00002/C00007/D00266/,female
freq,1,5,5,25,8192


In [7]:
train_df['gender'].value_counts()

female    8192
male      2308
Name: gender, dtype: int64

In [0]:
def get_no_of_prod(df):
  da, db, dc, dd = {}, {}, {}, {}
  no_prod = []
  for i in df['ProductList']:
    a = i.split(';')
    no_prod.append(len(a))
    dua, dub, duc = {}, {}, {}
    cna, cnb, cnc = 0, 0, 0
    for j in a:
      b = j[:-1].split('/')
      if(b[0] not in dua):
        dua[b[0]] = True
        cna+=1
      if(b[1] not in dub):
        dub[b[1]] = True
        cnb+=1
      if(b[2] not in duc):
        duc[b[2]] = True
        cnc+=1
      da[b[0]] = True
      db[b[1]] = b[0]
      dc[b[2]] = b[1]
      dd[b[3]] = b[2]
  print(len(da))
  print(len(db))
  print(len(dc))
  print(len(dd))
  df['no_of_prod_visited'] = no_prod
  return df



In [9]:
new_df = train_df.copy()
new_df = get_no_of_prod(new_df)
test_df = get_no_of_prod(test_df)

11
85
360
16503
11
71
282
8300


In [0]:
le = LabelEncoder()
new_df['gender'] = le.fit_transform(new_df['gender'])

In [11]:
new_df.head()

,session_id,startTime,endTime,ProductList,gender,no_of_prod_visited
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,0,4
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1,7
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,0,1
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,0,3
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1,2


In [0]:
def timestamp_proc(df):
  df['startTime'] = pd.to_datetime(df['startTime'])
  df['endTime'] = pd.to_datetime(df['endTime'])
  df['duration_of_sess'] = (df['endTime'] - df['startTime']).astype('timedelta64[m]')
  df['day'] = [i.weekday() for i in df['startTime']]
  df['hr'] = [i.hour for i in df['startTime']]
  return df

In [13]:
new_df = timestamp_proc(new_df)
test_df = timestamp_proc(test_df)
new_df.head()

,session_id,startTime,endTime,ProductList,gender,no_of_prod_visited,duration_of_sess,day,hr
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,0,4,1.0,0,18
1,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1,7,6.0,1,14
2,u19037,2014-01-12 15:58:00,2014-01-12 15:58:00,A00002/B00001/C00020/D16944/,0,1,0.0,6,15
3,u14556,2014-11-23 02:57:00,2014-11-23 03:00:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,0,3,3.0,6,2
4,u24295,2014-12-17 16:44:00,2014-12-17 16:46:00,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1,2,2.0,2,16


In [14]:
test_df.head()

,session_id,startTime,endTime,ProductList,no_of_prod_visited,duration_of_sess,day,hr
0,u12112,2014-08-12 13:36:00,2014-08-12 13:36:00,A00002/B00003/C00006/D19956/,1,0.0,1,13
1,u19725,2014-12-19 13:52:00,2014-12-19 13:52:00,A00002/B00005/C00067/D02026/,1,0.0,4,13
2,u11795,2014-01-12 10:44:00,2014-01-12 10:44:00,A00002/B00002/C00004/D12538/,1,0.0,6,10
3,u22639,2014-08-12 20:19:00,2014-08-12 20:22:00,A00002/B00003/C00079/D22781/;A00002/B00003/C00...,4,3.0,1,20
4,u18034,2014-12-15 19:33:00,2014-12-15 19:33:00,A00002/B00001/C00010/D23419/,1,0.0,0,19


In [0]:
y = new_df['gender']

In [16]:
to_drop = [ 'session_id', 'startTime', 'endTime', 'gender']
tr_df = new_df.drop(to_drop,axis=1)

test_id = pd.DataFrame({'session_id':test_df['session_id']})
to_drop_2 = [ 'session_id', 'startTime', 'endTime']
test_df = test_df.drop(to_drop_2,axis=1)
tr_df.head()

,ProductList,no_of_prod_visited,duration_of_sess,day,hr
0,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,4,1.0,0,18
1,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,7,6.0,1,14
2,A00002/B00001/C00020/D16944/,1,0.0,6,15
3,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,3,3.0,6,2
4,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,2,2.0,2,16


In [17]:
test_df.head()

,ProductList,no_of_prod_visited,duration_of_sess,day,hr
0,A00002/B00003/C00006/D19956/,1,0.0,1,13
1,A00002/B00005/C00067/D02026/,1,0.0,4,13
2,A00002/B00002/C00004/D12538/,1,0.0,6,10
3,A00002/B00003/C00079/D22781/;A00002/B00003/C00...,4,3.0,1,20
4,A00002/B00001/C00010/D23419/,1,0.0,0,19


In [0]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [19]:
def clean_prod_list(df):
  pl = []
  for text in df['ProductList']:
    tokens = tokenizer.tokenize(text)
    l = ''
    for t in tokens:
      if(t != '/' and t != '/;'):
        l += t + ' '
    pl.append(l)
  df['ProductList'] = pl
  return df

tr_df = clean_prod_list(tr_df)
test_df = clean_prod_list(test_df)
print(test_df.head())
print(tr_df.head())

                                         ProductList  ...  hr
0                       A00002 B00003 C00006 D19956   ...  13
1                       A00002 B00005 C00067 D02026   ...  13
2                       A00002 B00002 C00004 D12538   ...  10
3  A00002 B00003 C00079 D22781 A00002 B00003 C000...  ...  20
4                       A00002 B00001 C00010 D23419   ...  19

[5 rows x 5 columns]
                                         ProductList  ...  hr
0  A00002 B00003 C00006 D28435 A00002 B00003 C000...  ...  18
1  A00001 B00009 C00031 D29404 A00001 B00009 C000...  ...  14
2                       A00002 B00001 C00020 D16944   ...  15
3  A00002 B00004 C00018 D10284 A00002 B00004 C000...  ...   2
4  A00001 B00001 C00012 D30805 A00001 B00001 C000...  ...  16

[5 rows x 5 columns]


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
tmp_arr = cv.fit_transform(tr_df['ProductList']).toarray()
test_tmp_arr = cv.transform(test_df['ProductList']).toarray()
print(test_tmp_arr.shape)
print(tmp_arr.shape)

(4500, 16959)
(10500, 16959)


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [0]:
tmp_arr = scaler.fit_transform(tmp_arr)
test_tmp_arr = scaler.transform(test_tmp_arr)

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4000)

In [24]:
tmp = pca.fit_transform(tmp_arr)
test_tmp = pca.transform(test_tmp_arr)
print(test_tmp.shape)
print(tmp.shape)

(4500, 4000)
(10500, 4000)


In [25]:
tr_df = tr_df.drop(['ProductList'],axis=1)
test_df = test_df.drop(['ProductList'],axis=1)
print(test_df.head())
print(tr_df.head())

   no_of_prod_visited  duration_of_sess  day  hr
0                   1               0.0    1  13
1                   1               0.0    4  13
2                   1               0.0    6  10
3                   4               3.0    1  20
4                   1               0.0    0  19
   no_of_prod_visited  duration_of_sess  day  hr
0                   4               1.0    0  18
1                   7               6.0    1  14
2                   1               0.0    6  15
3                   3               3.0    6   2
4                   2               2.0    2  16


In [0]:
def get_time_spent(df): 
  nl = []
  for i,j in zip(df['no_of_prod_visited'], df['duration_of_sess']):
    if(int(j)!=0):
      nl.append( i/j )
    else:
      nl.append(j)
  df['time_spent'] = nl
  return df

tr_df = get_time_spent(tr_df)
test_df = get_time_spent(test_df)

In [27]:
ce_bin = ce.OneHotEncoder(cols = ['day'])
tr_df = ce_bin.fit_transform(tr_df)
test_df = ce_bin.transform(test_df)
print(test_df.head())
print(tr_df.head())

   no_of_prod_visited  duration_of_sess  day_1  ...  day_7  hr  time_spent
0                   1               0.0      0  ...      0  13    0.000000
1                   1               0.0      0  ...      0  13    0.000000
2                   1               0.0      0  ...      0  10    0.000000
3                   4               3.0      0  ...      0  20    1.333333
4                   1               0.0      1  ...      0  19    0.000000

[5 rows x 11 columns]
   no_of_prod_visited  duration_of_sess  day_1  ...  day_7  hr  time_spent
0                   4               1.0      1  ...      0  18    4.000000
1                   7               6.0      0  ...      0  14    1.166667
2                   1               0.0      0  ...      0  15    0.000000
3                   3               3.0      0  ...      0   2    1.000000
4                   2               2.0      0  ...      0  16    1.000000

[5 rows x 11 columns]


In [28]:
tmp2 = tr_df.values
tmp3 = np.concatenate((tmp2, tmp), axis=1)
train_arr = tmp3.astype(np.float32)
print(train_arr.shape)

test_tmp2 = test_df.values
test_arr = np.concatenate((test_tmp2, test_tmp), axis=1)
test_arr = test_arr.astype(np.float32)
print(test_arr.shape)

(10500, 4011)
(4500, 4011)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_arr, y, test_size=0.24,stratify=y, random_state=0)

In [30]:
X_train.shape

(7980, 4011)

In [0]:
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score as f1
from sklearn.metrics import make_scorer

In [32]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train,y_train)
y_pred = model_lr.predict(X_test)
print(f1(y_test,y_pred, average=None))
acc(y_test,y_pred)

[0.87037516 0.57361702]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8011904761904762

In [0]:
from sklearn.ensemble import RandomForestClassifier
import time

In [34]:
start = time.time()
model_rf = RandomForestClassifier(n_estimators=400, max_features=750, max_depth=20, n_jobs=-1, random_state=0)
model_rf.fit(X_train,y_train)
end = time.time()
y_pred = model_rf.predict(X_test)
print(f1(y_test,y_pred, average=None))
print(acc(y_test, y_pred))
print('Time taken : ' + str((end-start)/60))

[0.92530713 0.68659794]
0.8793650793650793
Time taken : 19.12809066772461


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1883,   83],
       [ 221,  333]])

In [36]:
y_pred = model_rf.predict(test_arr)
print(list(y_pred))

y_pred_cat = [ 'male' if i==1 else 'female' for i in y_pred ]
predictions = pd.DataFrame({'session_id':test_id['session_id'], 'gender':y_pred_cat})
predictions.head(10)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

,session_id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female
5,u11327,male
6,u12768,male
7,u17011,female
8,u13527,female
9,u24492,male


In [0]:
predictions.to_csv('Predictions_rf.csv', index=False)

In [0]:
from xgboost import XGBClassifier

In [39]:
start = time.time()
m = XGBClassifier(n_estimators=150, max_depth=18)
m.fit(X_train,y_train)
end = time.time()
y_pred = m.predict(X_test)
print(f1(y_test,y_pred, average=None))
print(acc(y_test, y_pred))
print('Time taken : ' + str((end-start)/60))

[0.92262198 0.6802444 ]
0.8753968253968254
Time taken : 17.739119569460552


In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1872,   94],
       [ 220,  334]])

In [41]:
y_pred = m.predict(test_arr)
print(list(y_pred))

y_pred_cat = [ 'male' if i==1 else 'female' for i in y_pred ]
predictions = pd.DataFrame({'session_id':test_id['session_id'], 'gender':y_pred_cat})
predictions.head(10)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

,session_id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female
5,u11327,male
6,u12768,male
7,u17011,female
8,u13527,female
9,u24492,male


In [0]:
predictions.to_csv('Predictions_xgb.csv', index=False)